In [1]:
import sys
sys.path.append("./expectation_value")
sys.path.append("./utils_hamiltonian")

from expectationvalue import ExpVal
from utils_hamiltonian import MoleculeHamiltonian, get_number_nbody_terms, get_obs_data
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
import qiskit.quantum_info as qif

import numpy as np
import h5py

In [3]:
molecules = {
    "H2":"H .0 .0 .0; H .0 .0 0.735",
    "LiH":"Li .0 .0 .0; H .0 .0 1.548",
    "BeH2": "Be .0 .0 .0; H .0 .0 -1.7; H .0 .0 1.7",
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH3": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

mol = 'H2'
# mol = 'LiH'
# mol = 'BeH2'
# mol = "H2O"
# mol = "NH3"

In [6]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits

bodies = get_number_nbody_terms(hamiltonian)

In [8]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()

print("Exact Ground State Energy: ", eig_gs)

-79.59454251461337

In [ ]:
# Load simulated parameters

# params_sim = h5py.File('data/params_2qubits.hdf5', 'r')
# lambda_k_t = params_sim["params"][:]


In [9]:
total_shots = 6000000
r = 30
reps = 100

In [10]:
# algorithm = ExpVal(n_shots = total_shots-1000,
#         bodies = bodies, 
#         r = r, 
#         r_shots = 1000,
#         n_qubits = n_qubits)

# algorithm.get_interferences(circ_gs)

# coeffs, obs = get_obs_data(hamiltonian)

# exp_val = algorithm.exp_val(obs)


# results = np.sum(exp_val*coeffs).real

In [11]:
results_god = []

for i in tqdm(range(reps)):
    algorithm = ExpVal(n_shots = total_shots-total_shots//4,
            bodies = bodies, 
            r = r, 
            r_shots = total_shots//4,
            n_qubits = n_qubits)

    algorithm.get_shadows_R(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)

    first_term = hamiltonian[0].coeffs[0].real

    print('Exp val calculado: ', np.sum(exp_val*coeffs).real + first_term)
    results_god.append(np.sum(exp_val*coeffs).real + first_term)

  1%|▍                                        | 1/100 [01:25<2:20:29, 85.15s/it]

Exp val calculado:  -79.67949963037745


  2%|▊                                        | 2/100 [02:50<2:18:53, 85.03s/it]

Exp val calculado:  -79.67901958151805


  3%|█▏                                       | 3/100 [04:14<2:17:20, 84.95s/it]

...

Exp val calculado:  -79.68083178357057


 99%|███████████████████████████████████████▌| 99/100 [2:19:33<01:23, 83.32s/it]

Exp val calculado:  -79.67833333729966


100%|███████████████████████████████████████| 100/100 [2:20:57<00:00, 84.58s/it]

Exp val calculado:  -79.67885497923038


In [12]:
print(eig_gs)

print(np.abs(results_god-eig_gs))

print(np.median(np.abs(results_god-eig_gs)))

-79.59454251461337
[0.08495712 0.08447707 0.08609546 0.0832267  0.08518683 0.08473566
 0.08504043 0.08587993 0.08530542 0.08571404 0.08725688 0.08568786
 0.08567286 0.08493349 0.08243772 0.0858856  0.08393683 0.08532206
 0.08530135 0.08277833 0.083249   0.08661382 0.08144629 0.08397533
 0.08705344 0.08708268 0.08243801 0.08600776 0.08338337 0.08634661
 0.08772059 0.08652751 0.08353997 0.08421132 0.08309616 0.08473194
 0.08719646 0.08688177 0.08473248 0.08410005 0.08553946 0.08591802
 0.08753691 0.08680562 0.08390263 0.08593093 0.08732775 0.08506715
 0.08344238 0.0856129  0.08404324 0.08262327 0.08486386 0.08563805
 0.08689925 0.08742767 0.08591742 0.08645939 0.08598688 0.08638492
 0.08608768 0.08563727 0.08567353 0.08470842 0.08948817 0.08445334
 0.08497981 0.08463926 0.08656109 0.08693976 0.08423662 0.08460093
 0.0835499  0.08370115 0.08711729 0.08534881 0.08483658 0.08506568
 0.08461668 0.08249531 0.0858049  0.08325641 0.08142758 0.08686654
 0.0835544  0.08718824 0.08482659 0.0851803

In [13]:
results = []


for i in tqdm(range(reps)):
    algorithm = ExpVal(n_shots = total_shots-1000,
            bodies = bodies, 
            r = r, 
            r_shots = 1000,
            n_qubits = n_qubits)

    algorithm.get_interferences(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)
    
    first_term = hamiltonian[0].coeffs[0].real

    print('Exp val calculado: ', np.sum(exp_val*coeffs).real + first_term)
    results.append(np.sum(exp_val*coeffs).real + first_term)

  1%|▍                                       | 1/100 [01:47<2:56:52, 107.20s/it]

Exp val calculado:  -79.58709473287345


  2%|▊                                       | 2/100 [03:46<3:06:19, 114.08s/it]

Exp val calculado:  -79.59461535241849


  3%|█▏                                      | 3/100 [05:42<3:06:30, 115.37s/it]

...

Exp val calculado:  -79.58770391943901


 99%|██████████████████████████████████████▌| 99/100 [3:02:55<01:51, 111.11s/it]

Exp val calculado:  -79.58648880499372


100%|██████████████████████████████████████| 100/100 [3:04:40<00:00, 110.81s/it]

Exp val calculado:  -79.57985725014356


In [14]:
print(eig_gs)

print(np.abs(results-eig_gs))

print(np.median(np.abs(results-eig_gs)))

-79.59454251461337
[7.44778174e-03 7.28378051e-05 1.43559224e-03 6.04523947e-03
 7.03984591e-03 5.22140899e-03 6.21060430e-03 5.71142706e-03
 6.16187546e-04 1.01319492e-02 4.48594387e-03 9.59659963e-03
 5.18591078e-03 1.02376574e-02 3.25796458e-03 7.46907754e-03
 3.92492511e-03 4.98478900e-03 7.13422399e-03 6.31936765e-03
 1.08313128e-02 8.29065827e-03 3.53319589e-03 4.55110728e-03
 6.55710570e-03 6.33938478e-03 1.22215862e-03 1.62720543e-03
 8.83973589e-03 6.42920920e-03 5.88066761e-03 6.64293228e-03
 6.09704823e-03 3.70331460e-03 7.99779550e-03 2.87678723e-03
 8.46211960e-03 5.67379334e-03 3.63834384e-03 7.84662538e-03
 6.97020902e-03 1.11831113e-02 8.37213305e-03 2.05265866e-03
 8.57888736e-03 7.70242690e-03 9.41474317e-03 2.51749276e-03
 4.54359548e-03 6.80241015e-03 7.92708591e-03 4.83592804e-03
 6.72140287e-03 5.36316617e-03 6.62182564e-03 2.81588229e-03
 6.39630238e-03 1.10940279e-02 2.16466145e-03 7.27533788e-03
 6.15959011e-03 6.53723304e-03 4.25415852e-03 5.18942377e-03
 7.53

In [15]:
num_paulis = len(hamiltonian)
shots_runtime = total_shots//num_paulis
shots_opflow = total_shots//num_paulis

In [16]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

#QiskitRuntimeService.save_account(channel="ibm_quantum", token="1a92dbd3ad8a3a75036e3b0fed7d2a10a0dfef2277960a619c033757aa1aa9bcc8f95922671a2fbd0b05339eea4aae6305bc43b5b2e6d33b161346f9983adf48")
service = QiskitRuntimeService(channel="ibm_quantum")

options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = shots_runtime

# with Session(service=service, backend="ibmq_qasm_simulator") as session:
#     estimator = Estimator(session=session, options = options)

#     job = estimator.run(circ_gs, hamiltonian)

#     result = job.result()

# result_runtime = result.values[0]
# np.abs(result_runtime-eig_gs)

In [17]:
results_runtime = []

for i in tqdm(range(rep)):
    
    with Session(service=service, backend="ibmq_qasm_simulator") as session:
        estimator = Estimator(session=session, options = options)

        job = estimator.run(circ_gs, hamiltonian)

        result = job.result()

    result_runtime = result.values[0]
    
    results_runtime.append(result_runtime)

NameError: name 'rep' is not defined

In [ ]:
print(eig_gs)

print(np.abs(results_runtime-eig_gs))

print(np.median(np.abs(results_runtime-eig_gs)))

In [ ]:
# from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
# from qiskit import Aer
# from qiskit.utils import QuantumInstance

# backend = Aer.get_backend("aer_simulator")
# quantum_instance = QuantumInstance(backend, shots=shots_opflow)

# op = hamiltonian
# psi = CircuitStateFn(circ_gs)

# measurable_expression = StateFn(op, is_measurement=True).compose(psi)
# expectation = PauliExpectation().convert(measurable_expression)  
# sampler = CircuitSampler(quantum_instance).convert(expectation)

# result_opflow = sampler.eval().real
# np.abs(result_opflow-eig_gs)